In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"

!pip install pandas-datareader==0.10.0

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# APIs - Práctica Guiada II

**Nota**: Esta notebook a veces no funciona por problemas en la API de yahoo.

Si aprece como error "No data fetched using 'YahooDailyReader'" prueben ejecutarla en otro momento.

<a id="section_toc"></a> 
## Tabla de Contenidos

[1- `pandas_datareader`](#section_pdr)

[2- Graficando la Serie de una acción](#section_series)

$\hspace{.5cm}$[2.1- Ejemplo: YPF](#section_ejemplo)

---

<a id="section_pdr"></a> 
###  1- `pandas_datareader`
[volver a TOC](#section_toc)

Para esta práctica usaremos [pandas_datareader](https://github.com/pydata/pandas-datareader), una librería que permite descargar directamente a los formatos (a esta altura muy conocidos) que ofrece Pandas.

La librería permite consumir datos de múltiples fuentes y armar DataFrames completos y con distintos formatos útiles. En esta práctica estudiaremos las series de distintas acciones de empresas conocidas, y estudiaremos gráficamente su comportamiento.

https://pydata.github.io/pandas-datareader/remote_data.html#remote-data-yahoo

In [ ]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data

plt.style.use('seaborn')

In [ ]:
# Si se precisa alguna otra acción, editar esta lista.
tickers = ['AAPL', 'MSFT', '^GSPC', 'PAMP.BA','PAM','YPF','YPFD.BA']

In [ ]:
start_date = '2010-01-01'
end_date = datetime.datetime.now().strftime('%Y%m%d')

In [ ]:
df = data.DataReader(tickers, data_source='yahoo', start=start_date, end=end_date)

Como podemos ver, obtenemos un DataFrame con MultiIndex en columnas, en el cual podemos ver distintos valores de precio para cada acción seleccionada (apertura, cierre, etc.). El formato es el de una serie de tiempo, con distintas posibles variables a regresionar (si quisiéramos hacerlo).

Para esta práctica, vamos a tomar como unidad de análisis el precio de cierre, de modo de quedar con un DataFrame cuya unidad observacional sea el cierre de un día, para cada una de las empresas observadas.

In [ ]:
close = df['Close']

In [ ]:
# freq='B' representa sólo los días de semana (business days).
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

close = close.reindex(all_weekdays)

close = close.fillna(method='ffill')

Observemos un poco cómo se distribuyen estos precios:

In [ ]:
close.describe()

<a id="section_series"></a> 
###  2- Graficando la Serie de una acción.
[volver a TOC](#section_toc)

Ahora vamos a elegir una empresa de las que observamos, y vamos a observar la serie de tiempo de sus precios de cierre. Para ello vamos a armar una pequeña función, que con la sigla de la empresa nos genere las visualizaciones que necesitamos.

In [ ]:
def graph_time_series(name):
    """
    Función para graficar los precios de cierre.
    
    Parameters
    ----------
    name : str
        Nombre de la acción a rastrear.

    """
    stock = close.loc[:, name]

    short_rolling_stock = stock.rolling(window=20).mean()
    long_rolling_stock = stock.rolling(window=100).mean()

    fig, ax = plt.subplots(figsize=(16,9))

    ax.plot(stock.index, stock, label=name)
    ax.set_title(f"Rolling means and actual value for stock {name}\n", fontsize=26)
    ax.tick_params(labelsize=14)
    ax.plot(short_rolling_stock.index, short_rolling_stock, label='20 days rolling')
    ax.plot(long_rolling_stock.index, long_rolling_stock, label='100 days rolling')
    ax.legend(fontsize=14)

<a id="section_ejemplo"></a> 
####  2.1- Ejemplo: YPF.
[volver a TOC](#section_toc)

Vamos a usar nuestra función para generar una visualización para una empresa.

**BONUS:** intentar usar la serie de alguna empresa para predecir los precios de la acción.

In [ ]:
graph_time_series("YPF")